# 📓 02 Análisis Exploratorio de Datos Espaciales (ESDA)

Este notebook se enfoca en la exploración visual y estadística de los datos cargados en PostGIS. Buscamos entender la distribución de las amenidades y validar la calidad de la información antes de aplicar modelos complejos.

---

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# --- CONFIGURACIÓN DE ENTORNO ---
load_dotenv("../.env")
db_url = f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST', 'localhost')}:5432/{os.getenv('POSTGRES_DB')}"
engine = create_engine(db_url)

print(f"Conexión establecida con la base de datos: {os.getenv('POSTGRES_DB')}")

## 1. Visualización de Capas Base

Comenzamos superponiendo la capa de amenidades (puntos) sobre los límites de la comuna para identificar patrones visuales iniciales.

In [2]:
# --- CARGA DE CAPAS DESDE POSTGIS ---
boundary = gpd.read_postgis("SELECT * FROM raw_data.comuna_boundaries", engine, geom_col='geometry')
amenities = gpd.read_postgis("SELECT * FROM raw_data.osm_amenities", engine, geom_col='geometry')

# --- MAPA EXPLORATORIO ---
fig, ax = plt.subplots(figsize=(10, 10))
boundary.plot(ax=ax, color='lightgrey', edgecolor='black', alpha=0.5, label='San Bernardo')
amenities.plot(ax=ax, markersize=8, color='red', alpha=0.4, label='Puntos de Servicio')
plt.title("Localización de Amenidades (OpenStreetMap)")
plt.show()

## 2. Análisis por Tipo de Amenidad

Queremos saber qué servicios son más frecuentes en San Bernardo. Esto nos ayuda a entender la vocación urbana de la zona.

In [3]:
# --- ESTADÍSTICAS DESCRIPTIVAS ---
# Extraemos los 15 tipos de amenidades más recurrentes
summary = amenities['amenity'].value_counts().head(15)

plt.figure(figsize=(12, 6))
sns.barplot(x=summary.values, y=summary.index, palette='magma')
plt.title("Frecuencia de Servicios Urbanos en San Bernardo")
plt.xlabel("Cantidad Detectada")
plt.ylabel("Categoría")
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

## 3. Mapas de Calor (Heatmaps)

Visualizar puntos individuales puede ser confuso. Un mapa de calor permite ver las áreas de mayor densidad acumulada.

In [4]:
import seaborn as sns

# Extraemos coordenadas X, Y de la geometría para el mapa de calor
amenities['x'] = amenities.geometry.x
amenities['y'] = amenities.geometry.y

plt.figure(figsize=(10, 10))
sns.kdeplot(x=amenities['x'], y=amenities['y'], fill=True, cmap='Blues', thresh=0.05, alpha=0.6)
boundary.boundary.plot(ax=plt.gca(), color='black', linewidth=1)
plt.title("Concentración Espacial de Servicios (Heatmap)")
plt.axis('off')
plt.show()